# Testing, Mocking and Python

##### Unit testing in Python and mocking techniques.

<img src="files/unittest.jpg" height="65%" width="65%" align="left">


# Introduction

## Me
- Darshan Ahluwalia
- Tutored Python at UC Irvine
- Using Python professionally for about 10 years
- I also like [gardening](http://husbandryandharvest.tumblr.com) and making [adventure shorts](https://vimeo.com/user38627724)

<img src="files/lost-arrow.jpg" align="left">


## Workshop?
- I'll ask questions
- You ask questions
- Laptop not really needed
- Its a jupyter notebook!
- [Source on GitHub](https://github.com/darshahlu/python_testing)

<img src="files/funny-horses.jpg" align="left" height="85%" width="85%">


## Outline

1. Unit Testing Principles
   1. What
   2. Why
   3. Rules
2. Mocking
   1. Monkey Patching
   2. Template Method (Inheritance)
   3. Dependency Injection
3. Testing in Python: HOWTO
   1. unittest
   2. doctest
   3. pytest
4. Real world example
   1. coverage
   2. write a test
   3. implement the feature

# Unit Testing Principles
##### What, why, and rules
<img src="files/monkey-blocks.jpg" height="55%" width="55%" align="left">


##### Let's define what is unit test by first looking at the code below.

In [17]:
def is_balanced(s):
    parens = {
        ')': '(',
        ']': '[',
        '}': '{',
    }
    opens = parens.values()
    closes = parens.keys()
    found_opens = list()
    for c in s:
        if c in opens:
            found_opens.append(c)
        elif c in closes:
            try:
                last_open = found_opens.pop()
            except IndexError:
                return False
            if parens[c] != last_open:
                return False
    return not len(found_opens)


# Do you do this? Is this testing your code? Is this Unit Test?
if __name__ == "__main__":
    print(is_balanced(''))
    print(is_balanced('hello'))


True
True


#### What is Unit Test?
- Software which tests software.
- Pieces of software, units, are tested in isolation.
- Automatable.

In [18]:
# This is unit test!

def test_is_balanced():
    assert is_balanced('') == True
    assert is_balanced('hello') == True
    assert is_balanced('(yes)') == True
    assert is_balanced('[(nope])') == False
    assert is_balanced('{}[]([wee!])') == True
    assert is_balanced('{') == False
    assert is_balanced(')') == False


##### Now let's talk about why we write tests. What does the is_balanced function do?

#### Why Unit Test?
- Requires you to write better code: more modular, more readable, easier to identify dependencies, etc.
- Documents the code in source: tests describe behaviors and functionality.
- Supports maintainance: adding features, refactoring, etc. (Did I break anything?)
- Finds issues early: during development.
- For onboarding developers: a sanity check that the environment is setup correctly.

##### Now let's talk about unit testing rules. What are some things one must consider when writing tests?

Consider testing software which interacts with a database. What are some problems you might encounter?

<img src="files/database.png" align="left">

#### Unit Testing Rules

Rules:

1. Must require no human interaction (e.g. do not ask for inputs).
2. Must be deterministic (i.e. always produce the same result).
3. Must be fast (sub-second up to few seconds).
4. Must be testing in isolation.

Guidelines:

1. Tests should be independent from each other (no side effects).
2. Write the fewest test cases to exercise the code.

#### Unit Testing Principles Summary
- What: software which test software.
- Why: reasons. (Software quality)
- Rules: fast, isolated, and automatable.

# Mocking
### What, why, and techniques

<img src="files/mocking.jpg" align="left">



#### What is mocking 
Mocking is the changing of software behavior to specific behaviors.

#### Why
Mocking is necessary to keep tests fast, isolated and with predictable behavior.

#### Techniques
The following techniques may be employed to mock out software components:
1. monkey patch
2. template method (inheritance/override behavior)
3. dependency injection

# The Monkey Patch 
##### A mocking technique using a monkey

<img src="files/monkey-tongue-1.jpg" height="55%" width="55%" align="left">


##### To talk about patching, we need to understand: objects and typing.

In [11]:
import math


class MyClass:
    """My class docstring"""
    A_CLASS_VAR = 1
    
    def __init__(self, a):
        self.a = a
    
    def some_method(self):
        pass

        
my_class_instance = MyClass(a='hi')


# What does this tell us about modules, classes and instances?
print('math is: ', type(math))
print('MyClass is: ', type(MyClass))
print('my_class_instance is: ', type(my_class_instance))
print()
print('math members are: ', dir(math))
print('MyClass members are: ', dir(MyClass))
print('my_class_instance members are: ', dir(my_class_instance))

math is:  <class 'module'>
MyClass is:  <class 'type'>
my_class_instance is:  <class '__main__.MyClass'>

math members are:  ['__doc__', '__loader__', '__name__', '__package__', '__spec__', 'acos', 'acosh', 'asin', 'asinh', 'atan', 'atan2', 'atanh', 'ceil', 'copysign', 'cos', 'cosh', 'degrees', 'e', 'erf', 'erfc', 'exp', 'expm1', 'fabs', 'factorial', 'floor', 'fmod', 'frexp', 'fsum', 'gamma', 'gcd', 'hypot', 'inf', 'isclose', 'isfinite', 'isinf', 'isnan', 'ldexp', 'lgamma', 'log', 'log10', 'log1p', 'log2', 'modf', 'nan', 'pi', 'pow', 'radians', 'sin', 'sinh', 'sqrt', 'tan', 'tanh', 'tau', 'trunc']
MyClass members are:  ['A_CLASS_VAR', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'some_method'

In [16]:
# Will this code execute? What does this tell us about Python's typing?
x = None
print("x is now: %s at %s" % (x, id(x)))
x = False
print("x is now: %s at %s" % (x, id(x)))
x = 1.2
print("x is now: %s at %s" % (x, id(x)))

x is now: None at 1484651424
x is now: False at 1484591008
x is now: 1.2 at 125206864


#### In Python, everything is an object and it is dynamically typed. 

What is monkey patching?  It is:

<img src="files/god-mode.jpg" align="left">


#### Definition
Dynamically change behaviors by reassigning attributes.

This can be on any object: an instance of an object, the class itself, a module or a package.

In [2]:
import time

class Timer(object):
    
    def sleep(self, s):
        print('sleeping for %i seconds...' % s)
        time.sleep(s)
        print('done sleeping')

timer = Timer()

def expensive_operation():
    print('performing expensive thingy')
    timer.sleep(4)
    print('done with it')

# We want to test this awesome code.
def some_awesome_code(a, b, c):
    expensive_operation()
    return sum([a, b, c])


In [3]:
# Expect 1 + 2 + 3 == 6
print(some_awesome_code(1, 2, 3))


performing expensive thingy
sleeping for 4 seconds...
done sleeping
done with it
6


In [4]:
# Let's make this fast by monkey patching the timer instance attribute.

def fast_sleep(seconds):
    print('waiting %i seconds.. not!' % seconds)
    return

timer.sleep = fast_sleep


In [5]:
some_awesome_code(1, 2, 3)


performing expensive thingy
waiting 4 seconds.. not!
done with it


6

#### More About Patching
- Patching can be on any object: an instance of an object, the class itself, a module or a package.
- patching the instance
``` timer.sleep = fast_sleep ```

- patching the Timer class
``` Timer.sleep = fast_sleep ```

- patching the module directly
``` time.sleep = fast_sleep ```

#### What's the difference?


 



#### Patching Rules

- Remember unittest guidelines: no side-effects (clean up after yourself)
```
original_sleep = time.sleep
time.sleep = fast_sleep
# ... do the tests
# finally:
time.sleep = original_sleep
```
- Just avoid this and patch the instance!


# Template method
#####  A mocking technique using inheritance and donuts

<img src="files/donut-inheritance.jpg" height="55%" width="55%" align="left">


##### What is one way we normally override behavior?

#### Definition
Change behaviors by inheriting from the class-under-test and overriding behavior.

This applies to classes only.

In [6]:
# Template method is only for classes, so refactor as such:

class MyTestableAwesomeCode(object):

    def _expensive_operation(self):
        print('performing expensive thingy')
        time.sleep(4)
        print('done with it')

    # We want to test this method
    def some_awesome_code(self, a, b, c):
        self._expensive_operation()
        return sum([a, b, c])

In [7]:
# Expect 1 + 2 + 3 == 6
m = MyTestableAwesomeCode()
print(m.some_awesome_code(1, 2, 3))

performing expensive thingy
done with it
6


In [8]:
# lets make it fast by inheriting from class-under-test and overriding the slow method.

class MyTestableAwesomeCodeFast(MyTestableAwesomeCode):
    
    def _expensive_operation(self):
        print('damn fast now')

In [9]:
m = MyTestableAwesomeCodeFast()
print(m.some_awesome_code(1, 2, 3))

damn fast now
6


##### What do you like about this method vs patching?  What do you dislike?

#### More About Classes
1. Use classes almost always for the benefit of code organization, OOP and...
2. For testability! Patch that instance or template it.
3. Multiple levels of inheritance is stinky (1-2 okay).


# Dependency Injection
##### It's like a flu shot without the side-effects

<img src="files/injection.jpg" height="50%" width="50%" align="left">


#### Definition

Fancytalk for "pass in objects as parameters" (so that you can provide objects with mocked behaviors later)

In [10]:
# requires refactoring: make dependencies inputs

import time

def expensive_operation():
    print('performing expensive thingy')
    time.sleep(4)
    print('done with it')

def some_awesome_code(a, b, c, expensive_operation=expensive_operation):
    expensive_operation()
    return sum([a, b, c])

In [11]:
# Expect 1 + 2 + 3 == 6
some_awesome_code(1, 2, 3)

performing expensive thingy
done with it


6

In [12]:
# Let's make this fast by providing a fake expensive operation
some_awesome_code(1, 2, 3, expensive_operation=lambda: None)


6

##### What do you like about this method over the others? Dislike?

# Patching vs Template Method vs Dependency Injection
##### Monkeys, donuts and injections
<img src="files/monkey-donut.jpg" height="60%" width="60%" align="left">

#### Patching
- Patching dynamically changes behavior by reassigning datamembers to new ones.
- Patching may be used on an object (luckily everything in Python is an object).
- Can only mock behavior that is an attribute of that object.
- Avoid patching globals like modules, packages, classes, if possible. Or restore the original behavior.

#### Template Method
- Template method statically defines new behavior (by creating a new class).
- Applicable to classes only.
- Can only mock behavior that is part of self.
- More code than patching due to typing out a new class.

#### Dependency Injection
- Fancytalk for "pass in objects as parameters"
- Requires you to provide dependencies as inputs.
- Dependencies are exposed in the interface.
- Less knowledge of class under test required.

# Testing in Python HOWTO
##### Using popular testing frameworks
<img src="files/goat-testing.jpg" align="left">


# Unittest (standard library)
- [Unittest](https://docs.python.org/2/library/unittest.html) is available in Python 2 and 3 standard library
- Unittest supports test automation by writing classes.

In [13]:
import unittest

def fib(n):
    """Return the n-th number in the fibonacci series."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    return fib(n-1) + fib(n-2)

class TestFib(unittest.TestCase):
    
    def test_fib_series(self):
        self.assertEqual(fib(0), 0)
        self.assertEqual(fib(1), 1)
        self.assertEqual(fib(2), 1)
        self.assertEqual(fib(3), 2)
        self.assertEqual(fib(4), 3)
        self.assertEqual(fib(5), 5)

if __name__ == "__main__":
    #normally this is unittest.main()...this is for jupyter
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)


test_fib_series (__main__.TestFib) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


# Doctest (standard library)
- [Doctest](https://docs.python.org/2/library/doctest.html) is available in Python 2 and 3 standard library
- Doctest allows you to write tests in your docstrings.

In [14]:
def fib(n):
    """Return the n-th number in the fibonacci series.
    
    >>> fib(0)
    0
    >>> fib(1)
    1
    >>> fib(29)
    514229
    """
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    return fib(n-1) + fib(n-2)


if __name__ == "__main__":
    import doctest
    #doctest.testmod()
    doctest.testmod(verbose=True)


Trying:
    fib(0)
Expecting:
    0
ok
Trying:
    fib(1)
Expecting:
    1
ok
Trying:
    fib(29)
Expecting:
    514229
ok
13 items had no tests:
    __main__
    __main__.MyTestableAwesomeCode
    __main__.MyTestableAwesomeCode._expensive_operation
    __main__.MyTestableAwesomeCode.some_awesome_code
    __main__.MyTestableAwesomeCodeFast
    __main__.MyTestableAwesomeCodeFast._expensive_operation
    __main__.TestFib
    __main__.TestFib.test_fib_series
    __main__.Timer
    __main__.Timer.sleep
    __main__.expensive_operation
    __main__.fast_sleep
    __main__.some_awesome_code
1 items passed all tests:
   3 tests in __main__.fib
3 tests in 14 items.
3 passed and 0 failed.
Test passed.


##### What is so awesome about doctest?

# Pytest (third party)
- [Pytest](https://docs.pytest.org/en/latest/) is available for Python 2.6+ and 3.3+ as a third-party package
- Combines doctest's simplicity and unittest's advanced features into a simple and powerful framework.
- Test discovery: for doctests, unittests and pytests.


In [16]:
# Running pytest in jupyter requires some magic.
# See: http://cprohm.de/article/ipytest-running-pytest-in-ipython-notebooks.html
__file__ = 'Python Testing.ipynb'
import ipytest.magics
import pytest

In [17]:
%%run_pytest[clean] -vvv

def fib(n):
    """Return the n-th number in the fibonacci series."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    return fib(n-1) + fib(n-2)

def test_fib_series():
    assert fib(0) == 0
    assert fib(1) == 1
    assert fib(2) == 1
    assert fib(3) == 2
    assert fib(4) == 3
    assert fib(5) == 5
    
@pytest.mark.parametrize('input,expected', ((29, 514229), (30, 832040), (31, 1346269)))
def test_fib_with_params(input, expected):
    assert fib(input) == expected


============================= test session starts =============================
platform win32 -- Python 3.6.3, pytest-3.5.0, py-1.5.3, pluggy-0.6.0 -- c:\dev\venvs\testing\scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\dev\repos\training\python_testing\jupyter_presentation, inifile: pytest.ini
plugins: django-3.1.2, cov-2.5.1
collecting ... collected 5 items

Python Testing.py::test_fib_series <- <ipython-input-17-924d9391d5e6> PASSED [ 20%]
Python Testing.py::test_fib_with_params[29-514229] <- <ipython-input-17-924d9391d5e6> PASSED [ 40%]
Python Testing.py::test_fib_with_params[30-832040] <- <ipython-input-17-924d9391d5e6> PASSED [ 60%]
Python Testing.py::test_fib_with_params[31-1346269] <- <ipython-input-17-924d9391d5e6> PASSED [ 80%]
Python Testing.py::TestFib::test_fib_series <- <ipython-input-13-37a129f9e642> PASSED [100%]

----------- coverage: platform win32, python 3.6.3-final-0 -----------
Name                         Stmts   Miss Branch BrPart  Cover
----------------

##### What is great about pytest over unittest?

# Setup and teardown

Both unittest and pytest support running the same code before and after every test.


#### Unittest setup and teardown

In [18]:
import traceback

class SomeTestCode(unittest.TestCase):
    
    def setUp(self):
        print('patching a global module')
        self.traceback_print_tb = traceback.print_tb
        traceback.print_tb = lambda *args, **kwargs: None
        
    def tearDown(self):
        print('unpatching')
        traceback.print_tb = self.traceback_print_tb
    
    def test_one(self):
        print('in test one')
    
    def test_two(self):
        print('in test two')
       
if __name__ == "__main__":
    #normally this is unittest.main()...this is for jupyter
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=0)


patching a global module
in test one
unpatching
patching a global module
in test two
unpatching


----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


#### Pytest setup and teardown

Pytest provides "fixtures" for setup and teardown.


In [19]:
%%run_pytest[clean] -qq

import pytest

@pytest.fixture(scope='function')
def mocked_module():
    print('patching')
    traceback_print_tb = traceback.print_tb
    yield
    print('unpatching')
    traceback.print_tb = lambda *args, **kwargs: None

def test_one(mocked_module):
    print('in test one')
    raise Exception("boo one!")
    
def test_two(mocked_module):
    print('in test two')
    raise Exception("boo two!")

# Note that if an exception happens during the setup code (before the yield keyword),
# the teardown code (after the yield) will not be called.

# A second pattern may be employed to ensure teardown happens even when
# setup fails (use a "finalizer")


============================= test session starts =============================
platform win32 -- Python 3.6.3, pytest-3.5.0, py-1.5.3, pluggy-0.6.0
rootdir: c:\dev\repos\training\python_testing\jupyter_presentation, inifile: pytest.ini
plugins: django-3.1.2, cov-2.5.1
collected 5 items

Python Testing.py FF...                                                  [100%]

================================== FAILURES ===================================
__________________________________ test_one ___________________________________
<ipython-input-19-0a1779c1cd32>:14: in test_one
    raise Exception("boo one!")
E   Exception: boo one!
---------------------------- Captured stdout setup ----------------------------
patching
---------------------------- Captured stdout call -----------------------------
in test one
-------------------------- Captured stdout teardown ---------------------------
unpatching
__________________________________ test_two ___________________________________
<ipython-input

# Frameworks Summary
##### Pytest vs Unittest vs Doctest
<img src="files/fight.jpg" align="left">



#### Unittest: 
- Familiar class-based JUnit-like syntax.
- Powerful enough to do all you need.

#### Doctest: 
- Tests are both documentation and actual tests!
- Great for simple tests (e.g. functions with simple inputs and outputs).

#### Pytest
- Finds and runs all your doctests, unittests and pytests.
- Does everything unittest does but in fewer lines of code.
- Great plugins: generate coverage, get a library of django fixtures, etc.




# Pytest vs Unittest
##### Pytest wins!
<img src="files/pytestvsdoctest.png" align="left">


# Real world example
##### It's getting real

- [pricing api tests](https://github.com/Aperio-Group/web-development/blob/feature/S-05326/WASAP/tests/views/api/tests_pricing_api.py)
- Pytest configuration: [pytest.ini](https://github.com/Aperio-Group/web-development/blob/feature/S-05326/pytest.ini)
- Coverage configuration: [.coveragerc](https://github.com/Aperio-Group/web-development/blob/feature/S-05326/.coveragerc)
- [Jenkins project](http://jenkins/job/wasap_dev/)

# Conclusion
- Write them **pytests, unittests and doctests.**
- Replace behaviors to make tests fast and isolated using **patching, inheritance and injection.**
- Automate them with **jenkins** and look at pretty graphs, **coverage** and stuff.

<img src="files/like-a-boss.jpg" align="left">




##### program like a boss
